In [19]:
! pip install 'git+https://github.com/nitkiew2/CMSE495-QSIDE-JUSTFAIR-TOOLBOX.git'

  Cloning https://github.com/nitkiew2/CMSE495-QSIDE-JUSTFAIR-TOOLBOX.git to /private/var/folders/lm/qgrqsck97r52_d1s83h76nkc0000gn/T/pip-req-build-4fpu64fr
  Running command git clone -q https://github.com/nitkiew2/CMSE495-QSIDE-JUSTFAIR-TOOLBOX.git /private/var/folders/lm/qgrqsck97r52_d1s83h76nkc0000gn/T/pip-req-build-4fpu64fr
  Resolved https://github.com/nitkiew2/CMSE495-QSIDE-JUSTFAIR-TOOLBOX.git to commit af188d3be2253ea034734f9a002732dc82997e44


In [20]:
import JUSTFAIR_Tools as jt
#from JUSTFAIR_Tools import toolbox

In [21]:
jt.toolbox.hello_world()

hello world!


In [23]:
jt.toolbox.matplotlib_test()

NameError: name 'plt' is not defined